In [24]:
import numpy as np
import pinocchio as pin
from nerf_grasping import kinematics_utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
urdf_filename = 'assets/trifinger/robot_properties_fingers/urdf/trifinger_with_stage.urdf'
model = pin.buildModelFromUrdf(urdf_filename)
data = model.createData()
q = pin.randomConfiguration(model)

frame_placement = pin.SE3.Random()
frame_type = pin.FrameType.OP_FRAME
kinematics_utils.add_ground_coll_frames(model)
model.addFrame(pin.Frame('test_frame', 4, 0, frame_placement, frame_type))
pin.forwardKinematics(model, data, q)
pin.updateFramePlacements(model, data)

for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}".format(name,
                                                     *oMi.translation.T.flat)))

universe                 :  0.00  0.00  0.00
finger_base_to_upper_joint_0 :  0.00  0.04  0.34
finger_upper_to_middle_joint_0 :  0.00  0.04  0.34
finger_middle_to_lower_joint_0 :  0.01  0.04  0.50
finger_base_to_upper_joint_120 :  0.03 -0.02  0.34
finger_upper_to_middle_joint_120 :  0.03 -0.02  0.34
finger_middle_to_lower_joint_120 :  0.07  0.13  0.29
finger_base_to_upper_joint_240 : -0.03 -0.02  0.34
finger_upper_to_middle_joint_240 : -0.03 -0.02  0.34
finger_middle_to_lower_joint_240 : -0.18 -0.01  0.40


In [3]:
fingertip_id = model.getFrameId('finger_tip_link_0')
T = pin.updateFramePlacement(model, data, fingertip_id)
T

  R =
    0.99748 0.000388948   0.0709485
          0   -0.999985  0.00548203
  0.0709496 -0.00546822   -0.997465
  p = -0.0191186 -0.0775661   0.608789

In [4]:
q = pin.randomConfiguration(model)
pin.framesForwardKinematics(model, data, q)
tip0 = model.getFrameId('finger_tip_link_0')
J = pin.computeFrameJacobian(model, data, q, tip0)
pin.updateFramePlacement(model, data, fingertip_id)

  R =
 0.951969  0.301018 0.0560599
        0 -0.183086  0.983097
 0.306194 -0.935878 -0.174292
  p =  0.0316362 -0.0277779   0.241642

In [19]:
q = pin.randomConfiguration(model)
tip_ids = kinematics_utils.get_tip_ids(model)

p_des = np.array([[
    0.0,
    0.10,
    0.05,
], [
    0.05,
    -0.10,
    0.05,
], [
    -0.05,
    -0.10,
    0.05,
]]).reshape(-1)

for ii in range(50):
    pin.forwardKinematics(model, data, q)
    J = kinematics_utils.get_fingertip_jacobian(model, data, q)
    p = np.concatenate([
        pin.updateFramePlacement(model, data, tt).translation for tt in tip_ids
    ],
                       axis=0)

    dq = -np.linalg.inv(J.T @ J + 1e-10 * np.eye(9)) @ J.T @ (p - p_des)

    q = q + 0.1 * dq

    print(np.linalg.norm(p_des - p))

0.6929150524849133
0.5606856305834215
0.5166100234308947
0.47419330258288966
0.43395599151605035
0.39601898582869
0.35967740176932395
0.31700829127237795
0.2961816047997018
0.27614900136966425
0.25682117663508297
0.23821875244539908
0.22043460422323535
0.2035857967614964
0.18777377288116553
0.17306425644637521
0.15948557557306786
0.14703903078477149
0.13572617665581901
0.1258167401354821
0.11607990398935793
0.10725430435747084
0.09927808053733442
0.09207459430347088
0.08556714017375097
0.07968461622072774
0.0743615900316012
0.06953780476212955
0.06515798227313484
0.06117177406987926
0.057533712602622133
0.05420309854127953
0.05114380835696952
0.04832402967461748
0.045715941548916825
0.04329535956749155
0.041041364785194934
0.03893593274451667
0.036963575323939325
0.03511100455474216
0.03336682426024476
0.03172125260801838
0.030165876496752946
0.028693437107977074
0.02729764487065578
0.025973021423234974
0.0247147658187053
0.023518642117033067
0.022380885572258268
0.02129812478980611


In [14]:
dq = np.random.randn(9).astype('float')
pin.forwardKinematics(model, data, q, dq)
fv = pin.getFrameVelocity(model, data, tip_ids[0])

In [13]:
ddq0 = np.zeros_like(dq)
b = pin.rnea(model, data, q, dq, ddq0)
M = pin.crba(model, data, q)

pin.forwardKinematics(model, data, q, dq)

print(type(q), type(dq))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
[ff.name for ff in model.frames]

In [23]:
tip_ids = kinematics_utils.get_tip_ids(model)
model.frames[tip_ids[0]].placement.translation += np.ones(3)

In [10]:
kinematics_utils.get_ground_coll_frame_ids(model)

[[39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58],
 [59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78],
 [79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98]]